# Test Models 

In [3]:
import sys
import os
from omegaconf import OmegaConf
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra

# Add the root directory to sys.path
sys.path.append(os.path.abspath('..'))
GlobalHydra.instance().clear()
initialize(version_base=None, config_path="../configs", job_name="test_app")
cfg = compose(config_name="config")

## Classifier

In [2]:
# imports

## Contrastive

In [10]:
from src.util.funcs import set_all_seeds, split_patients, create_version_name
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor, TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger
cfg.ml.normalize = False

In [11]:
set_all_seeds(cfg.seed)

all_cpus = set(range(os.cpu_count()))
# Set affinity to use all CPUs
os.sched_setaffinity(0, all_cpus)

feature_columns = [
'wrist_aff__acc_x', 'wrist_aff__acc_y', 'wrist_aff__acc_z',
'wrist_aff__gyro_x', 'wrist_aff__gyro_y', 'wrist_aff__gyro_z',
'wrist_nonaff__acc_x', 'wrist_nonaff__acc_y', 'wrist_nonaff__acc_z',
'wrist_nonaff__gyro_x', 'wrist_nonaff__gyro_y', 'wrist_nonaff__gyro_z'
] if cfg.ml.useGyro else [
'wrist_aff__acc_x', 'wrist_aff__acc_y', 'wrist_aff__acc_z',
'wrist_nonaff__acc_x', 'wrist_nonaff__acc_y', 'wrist_nonaff__acc_z',
]

train_patients, val_patients = split_patients(cfg)

version_name=create_version_name(cfg)

[rank: 0] Seed set to 42


### Base

In [12]:
from src.datasets.contrastiveDatasets import create_traindatasets, create_traindataloaders
from src.models.contrastiveModels import TimeSeriesSimCLR

In [13]:
train_dataset, val_dataset = create_traindatasets(cfg, train_patients, val_patients, feature_columns)
    
# Create dataloaders
train_loader, val_loader = create_traindataloaders(
    train_dataset, 
    val_dataset,
    batch_size=cfg.ml.batch_size,
    num_workers=len(os.sched_getaffinity(0))#cfg.ml.num_workers
)
# Initialize model
model = TimeSeriesSimCLR(
    input_dim=len(feature_columns),
    hidden_dim=cfg.ml.hidden_dim,
    projection_dim=cfg.ml.projection_dim,
    num_layers=cfg.ml.num_layers,
    temperature=cfg.ml.temperature,
    learning_rate=cfg.ml.learning_rate,
    dropout=cfg.ml.dropout,
    weight_decay=cfg.ml.weight_decay,
    batch_size=cfg.ml.batch_size,
)

In [14]:
# Setup callbacks
callbacks = [
    ModelCheckpoint(
        monitor='val_loss',
        dirpath=f"/cluster/work/vogtlab/Group/jamoser/checkpoints/{version_name}",
        filename='best_checkpoint',
        save_top_k=1,
        mode='min'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=20,
        mode='min'
    ),
    LearningRateMonitor(logging_interval='epoch'),
    TQDMProgressBar(refresh_rate=10)
]

# Setup logger
logger = TensorBoardLogger(
    save_dir="/cluster/work/vogtlab/Group/jamoser/lightning_logs",
    name=cfg.ml.logName,
    default_hp_metric=False,
    version=version_name
)

# Initialize trainer
trainer = pl.Trainer(
    max_epochs=100,
    accelerator='auto',
    devices='auto',
    # strategy='ddp',
    use_distributed_sampler=False,
    callbacks=callbacks,
    logger=logger,
    gradient_clip_val=1.0,
    gradient_clip_algorithm='norm',
    log_every_n_steps=10,
    deterministic=True,
    enable_progress_bar=True,
    enable_model_summary=True
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)
# logger.log_hyperparams({
#     'hidden_dim': cfg.ml.hidden_dim,
#     'projection_dim': cfg.ml.projection_dim,
#     'temperature': cfg.ml.temperature,
#     'num_layers': cfg.ml.num_layers,
#     'dropout': cfg.ml.dropout,
#     'learning_rate': cfg.ml.learning_rate,
#     'weight_decay': cfg.ml.weight_decay,
#     'noise_level': cfg.ml.noise_level,
#     'mask_ratio': cfg.ml.mask_ratio,
#     'batch_size': cfg.ml.batch_size,
#     'window_size': cfg.window_size,
#     'threshold': cfg.threshold_ratio
# },{
#     'hparam/loss': trainer.logged_metrics['val_loss_epoch'],
# })


  | Name       | Type       | Params | Mode 
--------------------------------------------------
0 | encoder    | LSTM       | 62.0 K | train
1 | projection | Sequential | 3.3 K  | train
--------------------------------------------------
65.2 K    Trainable params
0         Non-trainable params
65.2 K    Total params
0.261     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |                                                                                            …

Training: |                                                                                                   …


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

### Adversary

In [ ]:
from src.datasets.contrastiveDatasets import create_traindatasets, create_traindataloaders
from src.models.contrastiveModels import TimeSeriesSimCLRWithAdversary